In [18]:
import mediapipe as mp 
import numpy as np
import cv2

class MarjaryasanaDetector:
    """
    A class for detecting and analyzing the Marjaryasana (Cat Pose) yoga pose using MediaPipe.
    
    This class uses the MediaPipe Pose solution to detect and analyze body landmarks
    to determine if a person is correctly performing the Marjaryasana pose.
    
    Attributes:
        mp_pose: MediaPipe pose solution instance
        pose: MediaPipe pose detector configured for static images
    """
    
    def __init__(self):
        """
        Initialize the MarjaryasanaDetector with MediaPipe pose detection settings.
        """
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=True,  # Set to True for image processing (vs video)
            min_detection_confidence=0.5  # Minimum confidence threshold for pose detection
        )
        
    def calculate_angle(self, a, b, c):
        """
        Calculate the angle between three points in 2D space.
        
        Args:
            a (list): First point coordinates [x, y]
            b (list): Middle point coordinates [x, y] (vertex of angle)
            c (list): Last point coordinates [x, y]
            
        Returns:
            float: Angle in degrees between the three points (0-180)
        """
        a = np.array(a)
        b = np.array(b)
        c = np.array(c)
        
        # Calculate angle using arctangent
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - \
                  np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians*180.0/np.pi)
        
        # Ensure angle is between 0-180 degrees
        if angle > 180.0:
            angle = 360-angle
            
        return angle
    
    def get_point(self, landmark):
        """
        Convert a MediaPipe landmark to a 2D point.
        
        Args:
            landmark (mediapipe.framework.formats.landmark_pb2.NormalizedLandmark): 
                MediaPipe landmark object
                
        Returns:
            list: 2D point coordinates [x, y]
        """
        return [landmark.x, landmark.y]

    def detect_pose(self, image):
        """
        Detect if a person in the image is performing the Marjaryasana pose.
        
        The method checks several key characteristics of the pose:
        - Head position (tucked)
        - Back position (rounded)
        - Knee position (under hips)
        - Leg alignment (vertical)
        
        Args:
            image (numpy.ndarray): Input BGR image
            
        Returns:
            tuple: (is_marjaryasana, debug_info)
                - is_marjaryasana (bool): True if pose detected correctly
                - debug_info (dict): Dictionary containing detailed pose measurements
                  and intermediate checks
        """
        # Convert BGR to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.pose.process(image_rgb)
        
        if results.pose_landmarks is None:
            return False, "No pose detected"
            
        landmarks = results.pose_landmarks.landmark
        
        # Extract key anatomical landmarks for pose analysis
        nose = landmarks[self.mp_pose.PoseLandmark.NOSE]
        left_shoulder = landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[self.mp_pose.PoseLandmark.RIGHT_SHOULDER]
        left_hip = landmarks[self.mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = landmarks[self.mp_pose.PoseLandmark.RIGHT_HIP]
        left_knee = landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE]
        right_knee = landmarks[self.mp_pose.PoseLandmark.RIGHT_KNEE]
        left_ankle = landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE]
        right_ankle = landmarks[self.mp_pose.PoseLandmark.RIGHT_ANKLE]
        
        # Calculate key angles for pose verification
        spine_angle = self.calculate_angle(
            self.get_point(left_shoulder),
            self.get_point(left_hip),
            self.get_point(left_knee)
        )
        
        left_leg_angle = self.calculate_angle(
            self.get_point(left_hip),
            self.get_point(left_knee),
            self.get_point(left_ankle)
        )
        
        right_leg_angle = self.calculate_angle(
            self.get_point(right_hip),
            self.get_point(right_knee),
            self.get_point(right_ankle)
        )

        # Verify key pose characteristics
        head_tucked = nose.y > (left_shoulder.y + right_shoulder.y) / 2  # Check if head is below shoulders
        back_rounded = 90 <= spine_angle <= 130  # Verify back curvature
        knees_under_hips = (  # Check if knees are aligned under hips
            abs(left_knee.x - left_hip.x) < 0.1 and
            abs(right_knee.x - right_hip.x) < 0.1
        )
        legs_vertical = (  # Verify legs are approximately vertical
            75 <= left_leg_angle <= 100 and
            75 <= right_leg_angle <= 100
        )
        
        # Compile debug information for pose analysis
        debug_info = {
            "spine_angle": f"{spine_angle:.1f}°",
            "left_leg_angle": f"{left_leg_angle:.1f}°",
            "right_leg_angle": f"{right_leg_angle:.1f}°",
            "head_tucked": head_tucked,
            "back_rounded": back_rounded,
            "knees_under_hips": knees_under_hips,
            "legs_vertical": legs_vertical
        }
        
        # Final pose verification
        is_marjaryasana = (
            head_tucked and
            back_rounded and
            knees_under_hips and
            legs_vertical
        )
        
        return is_marjaryasana, debug_info

    def visualize_pose(self, image):
        """
        Draw pose landmarks and connections on the input image.
        
        Args:
            image (numpy.ndarray): Input BGR image
            
        Returns:
            numpy.ndarray: Image with pose landmarks and connections drawn
        """
        # Convert BGR to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.pose.process(image_rgb)
        
        if results.pose_landmarks:
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(
                image, 
                results.pose_landmarks,
                self.mp_pose.POSE_CONNECTIONS
            )
        
        return image

# Example usage of the MarjaryasanaDetector class
if __name__ == "__main__":
    detector = MarjaryasanaDetector()
    
    # Load and process test image
    image = cv2.imread("./input_images/pose3.jpg")
    if image is None:
        print("Error: Could not read image")
    else:
        # Detect pose and print results
        is_pose, details = detector.detect_pose(image)
        print(f"Is Marjaryasana: {is_pose}")
        print("Details:")
        for key, value in details.items():
            print(f"- {key}: {value}")
        
        # Generate and save annotated image
        annotated_image = detector.visualize_pose(image)
        cv2.imwrite("pose_annotated.jpg", annotated_image)

Is Marjaryasana: True
Details:
- spine_angle: 103.5°
- left_leg_angle: 89.7°
- right_leg_angle: 85.3°
- head_tucked: True
- back_rounded: True
- knees_under_hips: True
- legs_vertical: True
